Proof of <a class="ProveItLink" href="../../../../../../_theory_nbs_/theory.ipynb">proveit</a>.<a class="ProveItLink" href="../../../../../_theory_nbs_/theory.ipynb">physics</a>.<a class="ProveItLink" href="../../../../_theory_nbs_/theory.ipynb">quantum</a>.<a class="ProveItLink" href="../../theory.ipynb">QPE</a>.<a class="ProveItLink" href="../../theorems.ipynb#_fail_sum">_fail_sum</a> theorem
========

In [ ]:
import proveit
theory = proveit.Theory() # the theorem's theory

from proveit import defaults, Lambda
from proveit import a, b, c, e, f, l, m, x, A, B, C, N, Q, X, Omega
from proveit.logic import Forall, InSet, Union, Disjoint, Card
from proveit.logic.sets.functions.bijections import bijection_transitivity
from proveit.numbers import zero, Neg, Interval
from proveit.numbers.modular import (
    interval_left_shift_bijection)
from proveit.statistics import prob_of_disjoint_events_is_prob_sum
from proveit.physics.quantum.QPE import (
    _two_pow_t, _two_pow_t__minus_one, _b, _t_in_natural_pos, _two_pow_t_minus_one_is_nat_pos,
    _best_is_int, _Omega, _sample_space_def, _alpha_def,
    _full_domain, _neg_domain, _pos_domain, 
    _pos_domain_in_full_domain, _neg_domain_in_full_domain, 
    ModAdd, _sample_space_bijection, _Omega_is_sample_space, _outcome_prob_vs_best, _fail_def)

In [ ]:
%proving _fail_sum

In [ ]:
defaults.assumptions = (
    _fail_sum.condition,
    InSet(l, _full_domain))

In [ ]:
_alpha_def

In [ ]:
_sample_space_def

In [ ]:
_outcome_prob_vs_best.instantiate()

In [ ]:
fail_def_inst = _fail_def.instantiate()

In [ ]:
lower_bound = _neg_domain.lower_bound

In [ ]:
upper_bound = _pos_domain.upper_bound

In [ ]:
interval_card = Card(Interval(lower_bound, upper_bound)).computation()

In [ ]:
interval_left_shift_bijection

In [ ]:
lambda_map_bijection = interval_left_shift_bijection.instantiate(
    {a:lower_bound, b:upper_bound, c:_b, N:_two_pow_t, x:l})

In [ ]:
transformed_fail_def1 = fail_def_inst.inner_expr().rhs.transform(
    lambda_map_bijection.element, lambda_map_bijection.domain.domain, _Omega)

In [ ]:
b_modadd_l__def = ModAdd(_b, l).definition()

In [ ]:
transformed_fail_def2 = transformed_fail_def1.inner_expr().rhs.instance_expr.vert_expr_array[3][0].body.element.state.num.substitute(
    b_modadd_l__def.derive_reversed())
# Need to fix Lambda.global_repl so this works as well:
#transformed_fail_def2 = b_modadd_l__def.sub_left_side_into(transformed_fail_def1)
transformed_fail_def2

It would be nice to automate this next section:

In [ ]:
e_upper_bound = defaults.assumptions[0].derive_element_upper_bound()

In [ ]:
from proveit.numbers import one
ep1_upper_bound = e_upper_bound.derive_shifted(one)

In [ ]:
ep1_loose_upper_bound = ep1_upper_bound.add_right(one)

In [ ]:
ep1_upper_bound.derive_negated()

In [ ]:
from proveit.numbers import Add, NaturalPos, IntegerNeg
InSet(Add(e, one), NaturalPos).prove()

In [ ]:
InSet(Neg(Add(e, one)), IntegerNeg).prove()

In [ ]:
from proveit.numbers import Less
Less(_neg_domain.upper_bound, _pos_domain.lower_bound).prove()

In [ ]:
neg_and_pos_domain_are_disjoint = Disjoint(_neg_domain, _pos_domain).prove()

In [ ]:
prob_of_disjoint_events_is_prob_sum

In [ ]:
circuit_map = Lambda(l, transformed_fail_def2.rhs.instance_expr)

In [ ]:
condition = transformed_fail_def2.rhs.non_domain_condition()

In [ ]:
_pos_domain_in_full_domain.instantiate().instantiate(assumptions=[InSet(l, _pos_domain)])

In [ ]:
_neg_domain_in_full_domain.instantiate().instantiate(assumptions=[InSet(l, _neg_domain)])

In [ ]:
# from proveit.logic import Forall
# Forall(l, InSet(l, _full_domain), domain=Union(_pos_domain, _full_domain))#.prove()

In [ ]:
from proveit.logic import SubsetEq
SubsetEq(_pos_domain, _full_domain).prove()

In [ ]:
SubsetEq(_neg_domain, _full_domain).prove()

In [ ]:
SubsetEq(Union(_neg_domain, _pos_domain), _full_domain).prove()

Use

In [ ]:
lambda_map_bijection

In [ ]:
_sample_space_bijection

In [ ]:
b_modadd_l__def

In [ ]:
trans_bijection = lambda_map_bijection.apply_transitivity(
    _sample_space_bijection, replacements=[b_modadd_l__def.derive_reversed()])

In [ ]:
trans_bijection

In [ ]:
# trans_bijection.inner_expr().element.body.vert_expr_array[3][0].body.element.state.num.substitute(b_modadd_l__def.derive_reversed())

In [ ]:
# bijection_transitivity

In [ ]:
# prob_of_disjoint_events_is_prob_sum.instantiate(
#     {Omega:_Omega, A:_neg_domain, B:_pos_domain, C:Union(_neg_domain, _pos_domain), 
#      X:_full_domain, f:circuit_map, Q:Lambda(l, condition)}
# )